# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [187]:
import pandas as pd
import math

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

from scipy.stats import ks_2samp
import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy

import scipy.stats

%matplotlib inline

In [188]:
df = pd.read_csv('previsao_de_renda.csv')
df.drop(['Unnamed: 0', 'id_cliente'], axis=1, inplace=True)
df['data_ref'] = pd.to_datetime(df['data_ref'])
df['data_ref'] = df['data_ref'].dt.strftime('%m-%Y')

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [189]:
df['sexo'].value_counts(normalize=True)*100

F    67.46
M    32.54
Name: sexo, dtype: float64

In [190]:
df['posse_de_veiculo'].value_counts(normalize=True)*100

False    60.933333
True     39.066667
Name: posse_de_veiculo, dtype: float64

In [191]:
df['posse_de_imovel'].value_counts(normalize=True)*100

True     67.62
False    32.38
Name: posse_de_imovel, dtype: float64

In [192]:
df['qtd_filhos'].value_counts(normalize=True)*100

0     69.173333
1     20.246667
2      9.173333
3      1.233333
4      0.113333
7      0.033333
14     0.013333
5      0.013333
Name: qtd_filhos, dtype: float64

In [193]:
df['tipo_renda'].value_counts(normalize=True)*100

Assalariado         50.886667
Empresário          23.386667
Pensionista         17.213333
Servidor público     8.453333
Bolsista             0.060000
Name: tipo_renda, dtype: float64

In [194]:
df['educacao'].value_counts(normalize=True)*100

Secundário             59.300000
Superior completo      35.566667
Superior incompleto     3.860000
Primário                1.100000
Pós graduação           0.173333
Name: educacao, dtype: float64

In [195]:
df['estado_civil'].value_counts(normalize=True)*100

Casado      70.226667
Solteiro    11.986667
União        7.186667
Separado     5.860000
Viúvo        4.740000
Name: estado_civil, dtype: float64

In [196]:
df['tipo_residencia'].value_counts(normalize=True)*100

Casa             90.213333
Com os pais       4.500000
Governamental     3.013333
Aluguel           1.293333
Estúdio           0.553333
Comunitário       0.426667
Name: tipo_residencia, dtype: float64

In [197]:
df['idade'].value_counts(normalize=True)*100

40    3.586667
37    3.126667
43    3.053333
32    3.033333
33    2.940000
39    2.933333
38    2.920000
27    2.906667
60    2.900000
46    2.873333
30    2.860000
41    2.800000
28    2.733333
34    2.713333
44    2.686667
31    2.686667
42    2.680000
47    2.653333
36    2.560000
29    2.513333
50    2.493333
54    2.486667
35    2.480000
45    2.473333
51    2.360000
48    2.306667
52    2.260000
58    2.260000
53    2.220000
57    2.186667
59    2.053333
49    2.033333
56    1.966667
55    1.713333
61    1.686667
62    1.606667
63    1.586667
64    1.340000
26    1.180000
65    0.953333
25    0.886667
66    0.846667
24    0.660000
67    0.440000
23    0.173333
22    0.100000
68    0.086667
Name: idade, dtype: float64

In [198]:
df['tempo_emprego'].value_counts(normalize=True)*100

4.216438     0.305786
6.934247     0.241410
7.520548     0.233363
15.449315    0.217269
5.717808     0.209222
               ...   
31.202740    0.008047
5.279452     0.008047
0.652055     0.008047
2.827397     0.008047
2.726027     0.008047
Name: tempo_emprego, Length: 2589, dtype: float64

In [199]:
df['qt_pessoas_residencia'].value_counts(normalize=True)*100

2.0     54.540000
1.0     18.346667
3.0     17.006667
4.0      8.740000
5.0      1.193333
6.0      0.120000
9.0      0.033333
15.0     0.013333
7.0      0.006667
Name: qt_pessoas_residencia, dtype: float64

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.

- Utilizando os recursos do Patsy, coloque as variáveis qualitativas como dummies.
- Mantenha sempre a categoria mais frequente como casela de referência
- Avalie os parâmetros e veja se parecem fazer sentido prático.

In [200]:
y, x = patsy.dmatrices('np.log(renda) ~ data_ref + sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + tipo_renda + educacao + estado_civil + C(tipo_residencia, Treatment(1)) + idade + tempo_emprego + qt_pessoas_residencia', data = df)

In [201]:
sm.OLS(y, x).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.359
Model:                            OLS   Adj. R-squared:                  0.357
Method:                 Least Squares   F-statistic:                     182.5
Date:                Sun, 05 Feb 2023   Prob (F-statistic):               0.00
Time:                        18:55:09   Log-Likelihood:                -13554.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12388   BIC:                         2.748e+04
Df Model:                          38                                         
Covariance Type:            nonrobust                                         
=====================================================================================================================
                                                        coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------
Intercept                                             6.5319      0.231     28.306      0.000       6.080       6.984
data_ref[T.01-2016]                                  -0.0375      0.035     -1.057      0.291      -0.107       0.032
data_ref[T.02-2015]                                   0.0031      0.035      0.088      0.930      -0.066       0.073
data_ref[T.02-2016]                                  -0.0064      0.035     -0.181      0.856      -0.076       0.063
data_ref[T.03-2015]                                   0.0505      0.036      1.421      0.155      -0.019       0.120
data_ref[T.03-2016]                                  -0.0399      0.036     -1.115      0.265      -0.110       0.030
data_ref[T.04-2015]                                   0.0494      0.035      1.394      0.163      -0.020       0.119
data_ref[T.05-2015]                                  -0.0183      0.035     -0.518      0.605      -0.088       0.051
data_ref[T.06-2015]                                   0.0729      0.035      2.056      0.040       0.003       0.142
data_ref[T.07-2015]                                   0.0285      0.035      0.806      0.421      -0.041       0.098
data_ref[T.08-2015]                                   0.0010      0.035      0.027      0.978      -0.069       0.070
data_ref[T.09-2015]                                  -0.0072      0.035     -0.204      0.838      -0.076       0.062
data_ref[T.10-2015]                                   0.0231      0.035      0.650      0.516      -0.046       0.093
data_ref[T.11-2015]                                   0.0206      0.035      0.581      0.561      -0.049       0.090
data_ref[T.12-2015]                                   0.0720      0.036      2.021      0.043       0.002       0.142
sexo[T.M]                                             0.7882      0.015     53.787      0.000       0.759       0.817
posse_de_veiculo[T.True]                              0.0437      0.014      3.092      0.002       0.016       0.071
posse_de_imovel[T.True]                               0.0825      0.014      5.902      0.000       0.055       0.110
tipo_renda[T.Bolsista]                                0.2210      0.241      0.917      0.359      -0.252       0.694
tipo_renda[T.Empresário]                              0.1566      0.015     10.483      0.000       0.127       0.186
tipo_renda[T.Pensionista]                            -0.3122      0.241     -1.294      0.196      -0.785       0.161
tipo_renda[T.Servidor público]                        0.0575      0.022      2.587      0.010       0.014       0.101
educacao[T.Pós graduação]                             0.0980      0.159      0.616      0.538      -0.214       0.410
educacao[T.Secu

In [ ]:
# R. Os parâmetros parecem fazer sentido prático para o modelo.

2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


In [203]:
# Variável menos significante: data_ref (cim p-values acima de 5% para todas as variáveis dummies).

In [204]:
y, x = patsy.dmatrices('np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + tipo_renda + educacao + estado_civil + C(tipo_residencia, Treatment(1)) + idade + tempo_emprego + qt_pessoas_residencia', data = df)

In [205]:
sm.OLS(y, x).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     287.5
Date:                Sun, 05 Feb 2023   Prob (F-statistic):               0.00
Time:                        18:55:12   Log-Likelihood:                -13568.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12402   BIC:                         2.737e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
=====================================================================================================================
                                                        coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------
Intercept                                             6.5406      0.229     28.502      0.000       6.091       6.990
sexo[T.M]                                             0.7874      0.015     53.723      0.000       0.759       0.816
posse_de_veiculo[T.True]                              0.0441      0.014      3.119      0.002       0.016       0.072
posse_de_imovel[T.True]                               0.0829      0.014      5.926      0.000       0.055       0.110
tipo_renda[T.Bolsista]                                0.2209      0.241      0.916      0.360      -0.252       0.694
tipo_renda[T.Empresário]                              0.1551      0.015     10.387      0.000       0.126       0.184
tipo_renda[T.Pensionista]                            -0.3087      0.241     -1.280      0.201      -0.782       0.164
tipo_renda[T.Servidor público]                        0.0576      0.022      2.591      0.010       0.014       0.101
educacao[T.Pós graduação]                             0.1071      0.159      0.673      0.501      -0.205       0.419
educacao[T.Secundário]                               -0.0141      0.072     -0.196      0.844      -0.155       0.127
educacao[T.Superior completo]                         0.0938      0.072      1.298      0.194      -0.048       0.235
educacao[T.Superior incompleto]                      -0.0432      0.078     -0.555      0.579      -0.196       0.109
estado_civil[T.Separado]                              0.3265      0.111      2.935      0.003       0.108       0.545
estado_civil[T.Solteiro]                              0.2672      0.109      2.453      0.014       0.054       0.481
estado_civil[T.União]                                -0.0343      0.025     -1.361      0.174      -0.084       0.015
estado_civil[T.Viúvo]                                 0.3689      0.116      3.191      0.001       0.142       0.596
C(tipo_residencia, Treatment(1))[T.Aluguel]           0.0442      0.054      0.815      0.415      -0.062       0.150
C(tipo_residencia, Treatment(1))[T.Com os pais]       0.0184      0.029      0.625      0.532      -0.039       0.076
C(tipo_residencia, Treatment(1))[T.Comunitário]      -0.0754      0.091     -0.829      0.407      -0.254       0.103
C(tipo_residencia, Treatment(1))[T.Estúdio]           0.1107      0.084      1.319      0.187      -0.054       0.275
C(tipo_residencia, Treatment(1))[T.Governamental]    -0.0129      0.039     -0.331      0.741      -0.089       0.064
qtd_filhos                                           -0.2658      0.108     -2.463      0.014      -0.477      -0.054
idade                                                 0.0053      0.001      6.624      0.000       0.004       0.007
tempo_emprego  

In [ ]:
# R. Não houve alteração significativa, exceto pelo R² levemente menor que o inicial.

3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [206]:
# variáveis menos significantes são aquelas com p-value maior que 5%.

In [207]:
y, x = patsy.dmatrices('np.log(renda) ~ + sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + tipo_renda + educacao + estado_civil + C(tipo_residencia, Treatment(1)) + idade + tempo_emprego + qt_pessoas_residencia', data = df)

In [208]:
sm.OLS(y, x).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     287.5
Date:                Sun, 05 Feb 2023   Prob (F-statistic):               0.00
Time:                        18:55:13   Log-Likelihood:                -13568.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12402   BIC:                         2.737e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
=====================================================================================================================
                                                        coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------
Intercept                                             6.5406      0.229     28.502      0.000       6.091       6.990
sexo[T.M]                                             0.7874      0.015     53.723      0.000       0.759       0.816
posse_de_veiculo[T.True]                              0.0441      0.014      3.119      0.002       0.016       0.072
posse_de_imovel[T.True]                               0.0829      0.014      5.926      0.000       0.055       0.110
tipo_renda[T.Bolsista]                                0.2209      0.241      0.916      0.360      -0.252       0.694
tipo_renda[T.Empresário]                              0.1551      0.015     10.387      0.000       0.126       0.184
tipo_renda[T.Pensionista]                            -0.3087      0.241     -1.280      0.201      -0.782       0.164
tipo_renda[T.Servidor público]                        0.0576      0.022      2.591      0.010       0.014       0.101
educacao[T.Pós graduação]                             0.1071      0.159      0.673      0.501      -0.205       0.419
educacao[T.Secundário]                               -0.0141      0.072     -0.196      0.844      -0.155       0.127
educacao[T.Superior completo]                         0.0938      0.072      1.298      0.194      -0.048       0.235
educacao[T.Superior incompleto]                      -0.0432      0.078     -0.555      0.579      -0.196       0.109
estado_civil[T.Separado]                              0.3265      0.111      2.935      0.003       0.108       0.545
estado_civil[T.Solteiro]                              0.2672      0.109      2.453      0.014       0.054       0.481
estado_civil[T.União]                                -0.0343      0.025     -1.361      0.174      -0.084       0.015
estado_civil[T.Viúvo]                                 0.3689      0.116      3.191      0.001       0.142       0.596
C(tipo_residencia, Treatment(1))[T.Aluguel]           0.0442      0.054      0.815      0.415      -0.062       0.150
C(tipo_residencia, Treatment(1))[T.Com os pais]       0.0184      0.029      0.625      0.532      -0.039       0.076
C(tipo_residencia, Treatment(1))[T.Comunitário]      -0.0754      0.091     -0.829      0.407      -0.254       0.103
C(tipo_residencia, Treatment(1))[T.Estúdio]           0.1107      0.084      1.319      0.187      -0.054       0.275
C(tipo_residencia, Treatment(1))[T.Governamental]    -0.0129      0.039     -0.331      0.741      -0.089       0.064
qtd_filhos                                           -0.2658      0.108     -2.463      0.014      -0.477      -0.054
idade                                                 0.0053      0.001      6.624      0.000       0.004       0.007
tempo_emprego  

In [209]:
# Removendo as variáveis menos significativas (com p-value maior que 5%).

df['tipo_renda'] = df.tipo_renda.astype('category')
_df = df[(df.tipo_renda!='Bolsista') & (df.tipo_renda!='Pensionista')].copy(deep=True)
_df['tipo_renda'] = _df.tipo_renda.cat.remove_unused_categories()

In [210]:
_df['estado_civil'] = _df.estado_civil.astype('category')
_df = _df[(_df.estado_civil!='União')].copy(deep=True)
_df['estado_civil'] = _df.estado_civil.cat.remove_unused_categories()

In [211]:
_df['tipo_residencia'] = _df.tipo_residencia.astype('category')
_df = _df[(_df.tipo_residencia=='Casa')].copy(deep=True)
_df['tipo_residencia'] = _df.tipo_residencia.cat.remove_unused_categories()

In [212]:
y, x = patsy.dmatrices('np.log(renda) ~ + sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + tipo_renda + estado_civil + C(tipo_residencia) + idade + tempo_emprego + qt_pessoas_residencia', data = _df)
sm.OLS(y, x).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.359
Model:                            OLS   Adj. R-squared:                  0.358
Method:                 Least Squares   F-statistic:                     477.2
Date:                Sun, 05 Feb 2023   Prob (F-statistic):               0.00
Time:                        18:55:15   Log-Likelihood:                -11228.
No. Observations:               10231   AIC:                         2.248e+04
Df Residuals:                   10218   BIC:                         2.258e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept                          6.4964      0.236     27.554      0.000       6.034       6.959
sexo[T.M]                          0.8005      0.016     49.503      0.000       0.769       0.832
posse_de_veiculo[T.True]           0.0455      0.016      2.931      0.003       0.015       0.076
posse_de_imovel[T.True]            0.0892      0.016      5.715      0.000       0.059       0.120
tipo_renda[T.Empresário]           0.1779      0.016     10.803      0.000       0.146       0.210
tipo_renda[T.Servidor público]     0.0727      0.024      2.973      0.003       0.025       0.121
estado_civil[T.Separado]           0.3877      0.120      3.231      0.001       0.152       0.623
estado_civil[T.Solteiro]           0.3053      0.118      2.595      0.009       0.075       0.536
estado_civil[T.Viúvo]              0.4092      0.124      3.295      0.001       0.166       0.653
qtd_filhos                        -0.3108      0.117     -2.667      0.008      -0.539      -0.082
idade                              0.0045      0.001      5.253      0.000       0.003       0.006
tempo_emprego                      0.0617      0.001     54.527      0.000       0.059       0.064
qt_pessoas_residencia              0.3355      0.116      2.884      0.004       0.107       0.564
==============================================================================
Omnibus:                        1.897   Durbin-Watson:                   2.026
Prob(Omnibus):                  0.387   Jarque-Bera (JB):                1.920
Skew:                           0.032   Prob(JB):                        0.383
Kurtosis:                       2.980   Cond. No.                     2.11e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.11e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [214]:
# R. O modelo se comportou praticamente da mesma forma que o modelo inicial, inclusive com o mesmo valor para R².